In [11]:
import os
import mne
import torch
import scipy.io as sio
import h5py
import numpy as np

# If you intend to use GPU acceleration via PyTorch (optional)
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
print("Using device:", device)

Using device: mps


In [12]:
# Base directory for a experimental file
base_dir_one_raw = '/Users/rahul/PycharmProjects/Thesis-EEG/osfstorage-archive/task1 - NR/Raw data/YAC/YAC_NR1_EEG.mat'
base_dir_one_preprocessed =  '/Users/rahul/PycharmProjects/Thesis-EEG/osfstorage-archive/task1 - NR/Preprocessed/YAC/gip_YAC_NR1_EEG.mat'

In [13]:
# loading the data
def load_mat_file(file_path):
    """Load a .mat file using scipy.io.loadmat or h5py as a fallback."""
    try:
        # Try to load using scipy.io.loadmat
        mat = sio.loadmat(file_path, struct_as_record=False, squeeze_me=True)
        print(f"Loaded {file_path} using scipy.io.loadmat")
        return mat
    except NotImplementedError:
        # If loadmat fails (e.g., for MATLAB v7.3 files), use h5py
        try:
            mat = h5py.File(file_path, 'r')
            print(f"Loaded {file_path} using h5py.File")
            return mat
        except Exception as e:
            print(f"Error loading {file_path} with h5py: {e}")
            return None
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None

### Data analysis for RAW

In [14]:
mat_data = load_mat_file(base_dir_one_raw)
print(mat_data.keys())
print(mat_data['EEG'])

Loaded /Users/rahul/PycharmProjects/Thesis-EEG/osfstorage-archive/task1 - NR/Raw data/YAC/YAC_NR1_EEG.mat using scipy.io.loadmat
dict_keys(['__header__', '__version__', '__globals__', 'EEG'])


In [15]:
EEG = mat_data['EEG']
eeg_signals = EEG.data
srate = EEG.srate
chanlocs = EEG.chanlocs
event = EEG.event
chaninfo = EEG.chaninfo
print("This is one instance of EEG signnals",eeg_signals[0]) # one instance of EEG data
print("This is the sampling rate of the EEG signals",srate)
print("This is the chanlocs field of the EEG signals",chanlocs)
print("These are the events of the EEG signals",event)
print("This is the channel info for the eeg signals",chaninfo)

This is one instance of EEG signnals [-8608.2265625  -8590.71191406 -8534.28125    ... -8706.42871094
 -8764.57617188 -8797.41796875]
This is the sampling rate of the EEG signals 500
This is the chanlocs field of the EEG signals [<scipy.io.matlab._mio5_params.mat_struct object at 0x15d616530>
These are the events of the EEG signals [<scipy.io.matlab._mio5_params.mat_struct object at 0x15d28a7a0>
This is the channel info for the eeg signals <scipy.io.matlab._mio5_params.mat_struct object at 0x15d28ad10>


### Data analysis for Preprocessed

In [16]:
mat_data_preprocessed = load_mat_file(base_dir_one_preprocessed)
print(mat_data.keys())
print(mat_data['EEG'])

Loaded /Users/rahul/PycharmProjects/Thesis-EEG/osfstorage-archive/task1 - NR/Preprocessed/YAC/gip_YAC_NR1_EEG.mat using h5py.File
dict_keys(['__header__', '__version__', '__globals__', 'EEG'])


In [17]:
EEG_preprocessed = mat_data_preprocessed['EEG']
eeg_signals_preprocessed = EEG.data
srate_preprocessed = EEG.srate
chanlocs_preprocessed = EEG.chanlocs
event_preprocessed = EEG.event
chaninfo_preprocessed = EEG.chaninfo
print("This is one instance of EEG signnals",eeg_signals_preprocessed[0]) # one instance of EEG data
print("This is the sampling rate of the EEG signals",srate_preprocessed)
print("This is the chanlocs field of the EEG signals",chanlocs_preprocessed)
print("These are the events of the EEG signals",event_preprocessed)
print("This is the channel info for the eeg signals",chaninfo_preprocessed)

This is one instance of EEG signnals [-8608.2265625  -8590.71191406 -8534.28125    ... -8706.42871094
 -8764.57617188 -8797.41796875]
This is the sampling rate of the EEG signals 500
This is the chanlocs field of the EEG signals [<scipy.io.matlab._mio5_params.mat_struct object at 0x15d616530>
These are the events of the EEG signals [<scipy.io.matlab._mio5_params.mat_struct object at 0x15d28a7a0>
This is the channel info for the eeg signals <scipy.io.matlab._mio5_params.mat_struct object at 0x15d28ad10>


### Defined function to work on RAW data

In [18]:
# preprocessing the data
def allocated_data(file_path):
    mat_data = load_mat_file(file_path)
    # data initialisation
    EEG = mat_data['EEG']
    eeg_signals = EEG.data
    srate = EEG.srate
    chanlocs = EEG.chanlocs
    event = EEG.event
    chaninfo = EEG.chaninfo

    if eeg_signals is None:
        print("Could not extract EEG fields from",file_path)

    # Ensuring the EEG signals are a numpy array
    if not isinstance(eeg_signals, np.ndarray):
        eeg_signals = np.array(eeg_signals)

    # Transposing the signals if necessary as mne expects channels x samples
    if eeg_signals.shape[0] > eeg_signals.shape[1]:
        eeg_signals = eeg_signals.T

    # Instantiating additional metadata
    file_name = os.path.split(file_path)[1]

    # returnable data_entry
    data_entry = {
        'eeg_signals': eeg_signals,
        'sampling_rate': srate,
        'chanlocs': chanlocs,
        'event': event,
        'chaninfo': chaninfo,
        'file_name': file_name,
        'file_path': file_path
    }
    return data_entry

In [19]:
raw_data = allocated_data(base_dir_one_raw)
print(raw_data)

Loaded /Users/rahul/PycharmProjects/Thesis-EEG/osfstorage-archive/task1 - NR/Raw data/YAC/YAC_NR1_EEG.mat using scipy.io.loadmat
{'eeg_signals': array([[ -8608.2265625 ,  -8590.71191406,  -8534.28125   , ...,
         -8706.42871094,  -8764.57617188,  -8797.41796875],
       [ -9405.4765625 ,  -9389.46679688,  -9337.8828125 , ...,
         -7675.56103516,  -7746.24121094,  -7788.71191406],
       [-14141.72363281, -14125.93066406, -14068.42089844, ...,
        -13245.39257812, -13301.41894531, -13331.96582031],
       ...,
       [  3616.75366211,   3633.81494141,   3692.7980957 , ...,
          5068.80810547,   5051.31640625,   5012.67236328],
       [   671.68096924,    693.81964111,    761.52154541, ...,
          1509.6661377 ,   1455.50878906,   1421.46716309],
       [ -7872.82958984,  -7854.29394531,  -7785.07275391, ...,
         -6691.74902344,  -6755.09179688,  -6793.81396484]]), 'sampling_rate': 500, 'chanlocs': array([<scipy.io.matlab._mio5_params.mat_struct object at 0x15c

In [20]:
preprocessed_data = allocated_data(base_dir_one_preprocessed)

Loaded /Users/rahul/PycharmProjects/Thesis-EEG/osfstorage-archive/task1 - NR/Preprocessed/YAC/gip_YAC_NR1_EEG.mat using h5py.File


AttributeError: 'Group' object has no attribute 'data'